In [2]:
!pip install nltk --upgrade

Requirement already up-to-date: nltk in /usr/local/lib/python3.6/dist-packages (3.4.5)


In [0]:
#loading dataset
import pickle 
trainers=open('/content/traindump.txt','rb')
trainlist=pickle.load(trainers)
testers=open('/content/testdump.txt','rb')
testlist=pickle.load(testers)

In [0]:
#creating training tokens
tokens=[]
for i in trainlist:
  tokens.extend(i.split())
for i in testlist:
  tokens.extend(i.split())

In [0]:
#the training data will have a sequence of words followed by target word. We create sequences here of length 50 and 1 target word.
length = 51
sentences = []
for i in range(length, len(tokens)):
  sequences = tokens[i-length:i]
  line = ' '.join(sequences)
  sentences.append(line)
  #print(line)


In [6]:
import keras

Using TensorFlow backend.


In [0]:
#Class for vectorizing texts, or/and turning texts into sequences (=list of word indexes, where the word of rank i in the dataset (starting at 1) has index i).
tokenizer=keras.preprocessing.text.Tokenizer()

In [0]:
sentences

In [0]:
##creates a sort of word to index mapping. Then maps back the words to sequence indexes
tokenizer.fit_on_texts(sentences)
sentences = tokenizer.texts_to_sequences(sentences)

In [0]:
#0 is reserved for padding. So we need to add 1 for the embedding to map from 1 to vocab_size.
vocab_size = len(tokenizer.word_index) + 1

In [0]:
#creating the input sequence and the target word.
from numpy import array
from keras.utils import to_categorical
sentences = array(sentences)
X, y = sentences[:,:-1], sentences[:,-1]  # X is input sequences of words, y is target word
y = to_categorical(y, num_classes=vocab_size) #allocate index to word 
sent_length = X.shape[1]       

In [36]:
#creating the neural network
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=sent_length))  
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 50)            303500    
_________________________________________________________________
lstm_3 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 6070)              613070    
Total params: 1,067,470
Trainable params: 1,067,470
Non-trainable params: 0
_________________________________________________________________
None


In [37]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #categorical crossentropy as there are multiple classes of words to predict
# fit model
model.fit(X, y, batch_size=128, epochs=20)

Epoch 1/20
197725/197725 [==============================] - 334s 2ms/step - loss: 5.3409 - acc: 0.1795
Epoch 2/20
197725/197725 [==============================] - 329s 2ms/step - loss: 4.7463 - acc: 0.2302
Epoch 3/20
197725/197725 [==============================] - 330s 2ms/step - loss: 4.4963 - acc: 0.2499
Epoch 4/20
197725/197725 [==============================] - 330s 2ms/step - loss: 4.3298 - acc: 0.2589
Epoch 5/20
197725/197725 [==============================] - 330s 2ms/step - loss: 4.2076 - acc: 0.2659
Epoch 6/20
197725/197725 [==============================] - 332s 2ms/step - loss: 4.1084 - acc: 0.2704
Epoch 7/20
197725/197725 [==============================] - 329s 2ms/step - loss: 4.0265 - acc: 0.2740
Epoch 8/20
197725/197725 [==============================] - 328s 2ms/step - loss: 3.9537 - acc: 0.2783
Epoch 9/20
197725/197725 [==============================] - 328s 2ms/step - loss: 3.9051 - acc: 0.2811
Epoch 10/20
197725/197725 [==============================] - 331s 2ms/ste

In [0]:
#function for generating output
def create_output(model, tokenizer, seqlength, seed, number_words):
	gen = []
	text_input = seed
	for i in range(number_words):
		encoded = tokenizer.texts_to_sequences([text_input])[0]
		encoded = pad_sequences([encoded], maxlen=seqlength, truncating='pre')
		yhat = model.predict_classes(encoded, verbose=0)
		output = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				output = word
				break
		text_input += ' ' + output
		gen.append(output)
	return ' '.join(gen)

In [0]:
from keras.preprocessing.sequence import pad_sequences

In [0]:
length = 51
sent = []
for i in range(length, len(tokens)):
  sequences = tokens[i-length:i]
  line = ' '.join(sequences)
  sent.append(line)
  #print(line)

In [0]:
#omitting 1 word for prediction task
seqlength = len(sent[0].split()) - 1

In [0]:
#selecting random sentence for prediction
from random import randint
seed = sent[randint(0,len(sent))]

In [46]:
output_sentences = create_output(model, tokenizer,seqlength, seed, 12)
print(output_sentences)

have to do s s we have to do it s s


In [0]:
#We can see that as compared to RNNs, the output is coherent here also. It is also more coherent than ngrams as LSTMs can learn long term context much better than both RNNs and ngrams owing to the additional gates which they employ. HEre, due to lesser number of epochs, the output is not very good. Increasing the number of epochs will surely solve this issue.